This notebook provides the code to pair each PPP with a control paper ("similar paper") and to generate the files "loose_twins.tsv".

## Packages

In [1]:
## load packages 
from pySankey.sankey import sankey
import pandas as pd
import json, requests 
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from tqdm import tqdm
import json, requests 
import time
import unicodedata
from metaphone import doublemetaphone
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher
import re
import plotly.express as px
from math import radians, cos, sin, asin, sqrt
import datetime 
from datetime import date
import psycopg2

/home/fs01/spec1142/anaconda3/envs/patents2/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
## import database username and password
main_path = "/home/fs01/spec1142/Emma/PPPs/"

f = open(main_path + "database.txt", "r")
user , password = f.read().split()

In [3]:
## load PPP file 
PPPs = pd.read_csv(main_path + "PPPs_v2.tsv" , delimiter = "\t")

In [5]:
len(PPPs)

548315

## Getting all papers from the same journal

In [97]:
## getting all the PPPs journals and years the journals are publishing a PPP

def get_journal(i):

     """
    This function retrieves the publication date, venue or source, and concepts for each PPP in a list of papers from a PostgreSQL database, and stores the information in a dictionary. It also creates a separate dictionary to store the years in which each venue published papers. The dictionaries are then saved to a JSON file.

    Parameters:
    i (int): The starting index for selecting papers. (for multiprocessing)

    Note:
    - The function assumes that the `user`, `password`, `main_path`, `PPPs`, and `workers` variables are defined elsewhere in the code.
    - The function establishes a connection to a PostgreSQL database using the `psycopg2` library and executes a SQL query to fetch the required data.
    - The function stores the retrieved data in two dictionaries: `dic_journal`.
    - The `dic_journal` dictionary contains the years in which each venue published papers.
    - The function saves the `dic_journal` dictionary to a JSON file named "journals.json" in the specified directory.
    """

    workers = 1 
    
    dic_journal = {}   


    #establishing the connection with the database 
    conn = psycopg2.connect("user=" + user + " password=" + password) 
    cursor = conn.cursor()


    papers = list(set(PPPs["paper_id"].tolist()))
    index_papers = [ k for k in range(i,len(papers),workers)]

    # get papers date, journal 
    for k in index_papers:


        work_id = papers[k]


        text = """ SELECT  publication_date , venue_or_source 
                   FROM works_OpenAlex WHERE work_id = '"""+ work_id + """' 
                   UNION 
                   SELECT  publication_date , venue_or_source 
                   FROM works_OpenAlex3 WHERE work_id = '"""+ work_id + """';"""


        cursor.execute(text)  
        res = cursor.fetchall()

        if len(res) > 0: 
            line = res[0]

            if line[1] != '':

                venue = line[1]   
                if line[0] != None:
                    year = line[0].year
                    if venue not in dic_journal: 
                        dic_journal[venue] = set()
                    dic_journal[venue].add(year)

    
    for venue in dic_journal:
        dic_journal[venue] = list(dic_journal[venue])

    # save journal and their corresponding papers 
    import json
    json = json.dumps(dic_journal)
    f = open(main_path + "PPP_analysis/journals.json","w")
    f.write(json)
    f.close()
    
    
    
get_journal(0) 

In [107]:
## file with all PPP journals and years the journals are publishing a PPP

f = open(main_path + "PPP_analysis/journals.json","r")
import json
dic_journals = json.load(f)

In [76]:
## for each journal, get all papers, publication years and concepts. 

def get_papers_journal(i):
    """
    This function retrieves the publication date and concepts for each paper published in a list of journals from a PostgreSQL database, and stores the information in a dictionary. The dictionary is then saved to a JSON file.

    Parameters:
    i (int): The starting index for selecting journals.

    Note:
    - The function assumes that the `user`, `password`, `main_path`, `dic_journals`, and `workers` variables are defined elsewhere in the code.
    - The function establishes a connection to a PostgreSQL database using the `psycopg2` library and executes a SQL query to fetch the required data.
    - The function stores the retrieved data in a dictionary `dic_papers_journals`, which contains the publication date and concepts for each paper published in each journal.
    - The function saves the `dic_papers_journals` dictionary to a JSON file named "dic_papers_journal_i.json" in the specified directory, where i is the starting index for selecting journals.
    - The function closes the database connection after fetching the data.
    """

    workers =  1

    dic_papers_journals = {} 

    #establishing the connection with the database 
    conn = psycopg2.connect("user=" + user + " password=" + password) 
    cursor = conn.cursor()


    list_journals = list(dic_journals.keys())
    index_journal = [ k for k in range(i,len(list_journals),workers)]

    ## for each journal, get works, publication date, concepts 
    for k in index_journal:


        journal = list_journals[k]
        dic_papers_journals[journal] = {}


        text = """ SELECT w.work_id , w.publication_date , w.concepts 
                   FROM works_OpenAlex AS w
                   WHERE venue_or_source = '"""+ journal + """';"""
                   

        cursor.execute(text)  
        res = cursor.fetchall()

        for line in res:
            work_id = line[0]
            if work_id not in dic_papers_journals[journal]:
                dic_papers_journals[journal][work_id] = {}
                if line[1] != None:
                    dic_papers_journals[journal][work_id]["date"] = line[1].strftime('%m-%d-%Y')
                else:
                    dic_papers_journals[journal][work_id]["date"] = None


                if line[2] != None:
                    concepts = set(line[2].replace("\r","").split("#tab#"))
                else:
                    concepts = set()
                dic_papers_journals[journal][work_id]["concepts"]  = ", ".join(list(concepts))
                
    
    ## save the data with works, date and concepts for each journal
    import json
    json = json.dumps(dic_papers_journals)
    f = open(main_path + "PPP_analysis/dic_papers_journal_" + str(i) + ".json","w")
    f.write(json)
    f.close()
    


get_papers_journal(0)

## Identify most similar paper

In [69]:
## file with all papers published in the same year and journal as the PPPs. 

f = open(main_path + "PPP_analysis/dic_papers_journal.json","r")
import json 
dic_papers_journals = json.load(f)

In [94]:
## group the papers published in the same journal by year. 

keys = list(dic_papers_journals.keys())
for k in tqdm(range(12,len(dic_papers_journals))):
    journal = keys[k]
    dic = {}
    for paper in dic_papers_journals[journal]:
        if dic_papers_journals[journal][paper]['date'] != None:
            paper_year = int(dic_papers_journals[journal][paper]['date'][-4:])
            if paper_year not in dic:
                dic[paper_year] = {}
            dic[paper_year][paper] = dic_papers_journals[journal][paper]
    dic_papers_journals[journal] = dic

100%|██████████████████████████████████████| 8580/8580 [01:21<00:00, 105.27it/s]


In [109]:
## for each PPP paper published by the journal, select up to 100 random papers published the same year in the same journal 

import random 

dic_papers_same_journal_year = {}
set_papers_PPPs = set(list(PPPs['paper_id']))

for journal in tqdm(dic_papers_journals):

    for year in dic_papers_journals[journal]:
        set_papers_journal = set(list(dic_papers_journals[journal][year].keys()))
        overlap = set_papers_journal & set_papers_PPPs

        for paper in overlap:
      
            dic_papers_same_journal_year[paper] = {}
            dic_papers_same_journal_year[paper]['data'] = dic_papers_journals[journal][year][paper]
            dic_papers_same_journal_year[paper]['papers'] = []

    
            for other_paper in set_papers_journal:
                if other_paper != paper_year:
                    dic_papers_same_journal_year[paper]['papers'].append(other_paper)
                if len(dic_papers_same_journal_year[paper]['papers']) >= 100:
                    break
        
        
        

100%|██████████████████████████████████████| 8592/8592 [00:35<00:00, 244.66it/s]


In [137]:
## save up to 100 random papers published the same year in the same journal 

import json
json = json.dumps(dic_papers_same_journal_year)
f = open(main_path + "PPP_analysis/dic_papers_same_journal_year.json","w")
f.write(json)
f.close()

In [ ]:
## for each PPP, select the most similar paper published the same year and in the same journal that is NOT a PPP. 

run PPP_analysis/control_papers.py 

In [158]:
## merge files - df_twins is the PPP twins file 

df_twins = pd.concat( [ pd.read_csv(main_path + "PPP_analysis/similar_papers_" + str(i) + ".tsv" , sep = "\t") for i in range(24) ]) 
df_twins.to_csv(main_path + "PPP_analysis/similar_papers.tsv", sep ="\t", index = False)

## Additional data on the PPPs / paired paper

In [61]:
## load PPPs

PPPs = pd.read_csv(main_path + "PPPs_v2.tsv" , delimiter = "\t")
set_paper_id = set(list(PPPs['paper_id']))

### Papers dates

In [62]:
## load twins (similar paper)

loose_twins = pd.read_csv(main_path + "PPP_analysis/similar_papers.tsv" , delimiter = "\t", index_col = 0 )

In [63]:
## query the publication date of the PPP and the twin (similar paper)

def get_dates_twins(i):

    """
    This function retrieves the publication dates of twin papers from the OpenAlex database and stores them in a dictionary.

    Parameters:
    i (int): The starting index for selecting twin papers.

    Note:
    - The function assumes that the `user`, `password`, `loose_twins`, and `workers` variables are defined elsewhere in the code.
    - The function establishes a connection to a PostgreSQL database using the `psycopg2` library and executes SQL queries to fetch the required data.
    - The function stores the publication dates of twin papers in a dictionary `dic_dates`.
    - The function returns the `dic_dates` dictionary.
    """

    #establishing the connection with the database 
    conn = psycopg2.connect("user=" + user + " password=" + password) 
    cursor = conn.cursor()

    dic_dates = {}

    workers = 96

    papers1 = list(set(list(loose_twins.index)))
    papers2 = list(set(list(loose_twins['twin'])))
    
    index_papers1 = [ k for k in range(i,len(papers1), workers)]
    index_papers2 = [ k for k in range(i,len(papers2), workers)]

    ## get publication date of the PPPs
    for k in index_papers1:
        
        work_id = papers1[k]
        
        dic_dates[work_id] = {}
        
        text = """ SELECT  w.publication_date
               FROM works_OpenAlex AS w 
               WHERE w.work_id = '"""+ work_id + """';"""
    
        
        cursor.execute(text)
        res = cursor.fetchall()
                
        if len(res) > 0:
            dic_dates[work_id]['paper_date'] = res[0][0]
        else:
            dic_dates[work_id]['paper_date'] = None


    ## get publication date of the twins 
    for k in index_papers2:
        
        work_id = papers2[k]
        if pd.isna(work_id) == False:
        
            dic_dates[work_id] = {}
            
            text = """ SELECT  w.publication_date
                   FROM works_OpenAlex AS w 
                   WHERE w.work_id = '"""+ work_id + """';"""
        
            
            cursor.execute(text)
            res = cursor.fetchall()
                    
            if len(res) > 0:
                dic_dates[work_id]['paper_date'] = res[0][0]
            else:
                dic_dates[work_id]['paper_date'] = None
        else:
            dic_dates[work_id] = {}
            dic_dates[work_id]['paper_date'] = None
            

    return dic_dates



In [64]:
## run the function get_dates_twins using 96 cpus

from multiprocessing import Pool
from functools import partial

p = Pool(96)
func = partial(get_dates_twins)
dates = p.map(func, [ i  for i in range(96)])
p.close()

## merge results of the threads
dic_dates = {} 
for elem in dates:
    dic_dates = { **dic_dates , **elem}
print(len(dic_dates))

### Patents dates

In [73]:
## query the patent application id, grant date and application date date of the patent PPP. 

def get_dates_patents(i):

    """
    This function retrieves the patent dates, application IDs, and application dates of patents from the PatentsView database and stores them in a dictionary.

    Parameters:
    i (int): The starting index for selecting patents.

    Note:
    - The function assumes that the `user`, `password`, `PPPs`, and `workers` variables are defined elsewhere in the code.
    - The function establishes a connection to a PostgreSQL database using the `psycopg2` library and executes SQL queries to fetch the required data.
    - The function stores the patent dates, application IDs, and application dates of patents in a dictionary `dic_dates`.
    - The function returns the `dic_dates` dictionary.
    """

    #establishing the connection with the database 
    conn = psycopg2.connect("user=" + user + " password=" + password) 
    cursor = conn.cursor()

    dic_dates = {}

    workers = 12

    patents = list(set(list(PPPs['patent_id'])))
    
    index_patent = [ k for k in range(i,len(patents), workers) ]

    
    # get the application/grant date of the patents
    for k in index_patent:
        
        patent_id = patents[k][3:]
        
        dic_dates[patent_id] = {}
        
        text = """ SELECT  p.patent_date , a.application_id, a.filing_date
               FROM patents_PatentsView AS p 
               LEFT JOIN applications_PatentsView AS a ON p.patent_id = a.patent_id
               WHERE p.patent_id = '"""+ str(patent_id) + """';"""
    
        cursor.execute(text)
        res = cursor.fetchall()
                
        if len(res) > 0:
            dic_dates[patent_id]['patent_date'] = res[0][0]
            dic_dates[patent_id]['application_id'] = res[0][1]
            dic_dates[patent_id]['application_date'] = res[0][2]
        else:
            dic_dates[patent_id]['patent_date'] = None
            dic_dates[patent_id]['application_id'] = None
            dic_dates[patent_id]['application_date'] = None
    
    return dic_dates


In [74]:
## run the function get_dates_patents using 12 cpus

from multiprocessing import Pool
from functools import partial

p = Pool(12)
func = partial(get_dates_patents)
dates = p.map(func, [ i  for i in range(12)])
p.close()


## merge results of the threads
dic_dates_patents = {} 
for elem in dates:
    dic_dates_patents = { **dic_dates_patents , **elem}
print(len(dic_dates_patents))

### Add pair id, dates and confidence score to the file and save file

In [76]:
## merge twin file and PPPs

data = loose_twins.merge(PPPs[['paper_id','patent_id', 'PPP_score']], left_index = True, right_on = 'paper_id', how = 'outer')
data

,date,twin,score,paper_id,patent_id,PPP_score
329497,07-15-2015,W2056326110,0.427760,W1578801066,US-9786832,1
110398,06-20-2006,W582671889,0.633810,W2249867704,US-8778608,2
532744,03-17-2015,W2002262929,0.427257,W2040274814,US-11099263,2
332594,03-22-2010,W2053503785,0.445780,W2022201375,US-8394762,4
304587,03-01-1988,W2064598121,0.551622,W1972554422,US-4990291,3
...,...,...,...,...,...,...
286001,1991-09-01,W4242375445,0.199821,W4245219720,US-5191616,1
286006,1991-09-01,W4242375445,0.199821,W4245219720,US-5009281,1
286013,1991-09-01,W4242375445,0.199821,W4245219720,US-4987564,1
286020,1991-09-01,W4242375445,0.199821,W4245219720,US-4989187,1


In [77]:
## generate flat file with all the relevant info 

pair = 0 
k = 0 
dic_loose_twins = {}

np_data = data[['paper_id', 'twin', 'patent_id', 'score','PPP_score' ]].to_numpy()

for elem in tqdm(np_data): 
    paper_id , twin , patent_id , score,PPP_score  = elem
    
    dic_loose_twins[k] = {}
    dic_loose_twins[k]['paper_id'] = paper_id
    dic_loose_twins[k]['patent_id'] = patent_id
    dic_loose_twins[k]['pair_id'] = pair
    dic_loose_twins[k]['twin_score'] = score
    dic_loose_twins[k]['PPP'] = 1
    dic_loose_twins[k]['PPP_score'] = PPP_score
    dic_loose_twins[k]['paper_date'] = dic_dates[paper_id]['paper_date']
    dic_loose_twins[k]['patent_date'] = dic_dates_patents[patent_id[3:]]['patent_date']
    dic_loose_twins[k]['application_id'] = dic_dates_patents[patent_id[3:]]['application_id']
    dic_loose_twins[k]['application_date'] = dic_dates_patents[patent_id[3:]]['application_date']
    k += 1

    dic_loose_twins[k] = {}
    dic_loose_twins[k]['paper_id'] = twin
    dic_loose_twins[k]['patent_id'] = patent_id
    dic_loose_twins[k]['pair_id'] = pair
    dic_loose_twins[k]['twin_score'] = score
    dic_loose_twins[k]['PPP'] = 0
    dic_loose_twins[k]['PPP_score'] = PPP_score
    if pd.isna(twin) == False:
        dic_loose_twins[k]['paper_date'] = dic_dates[twin]['paper_date']
    else:
        dic_loose_twins[k]['paper_date'] = None
    dic_loose_twins[k]['patent_date'] = dic_dates_patents[patent_id[3:]]['patent_date']
    dic_loose_twins[k]['application_id'] = dic_dates_patents[patent_id[3:]]['application_id']
    dic_loose_twins[k]['application_date'] = dic_dates_patents[patent_id[3:]]['application_date']
    k += 1
    pair += 1
    
df_loose_twins = pd.DataFrame(dic_loose_twins).T
df_loose_twins

100%|████████████████████████████████| 548315/548315 [00:44<00:00, 12297.75it/s]


In [ ]:
## add patent's publication date (if exists) 

publication_dates = pd.read_csv(main_path + "published_application_small.tsv", index_col = 0 )
publication_dates = publication_dates.drop_duplicates('application_id')
df_loose_twins = df_loose_twins.merge(publication_dates, on ='application_id', how='left')

In [81]:
## save the twin file 

df_loose_twins.to_csv(main_path + "PPP_analysis/loose_twins.tsv", index = False, sep = "\t")